In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd
import librosa
import librosa.display
import scipy.io.wavfile as wavf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
from keras.preprocessing.sequence import pad_sequences
import copy
from numpy import dot
from numpy.linalg import norm
import operator

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd
import librosa
import librosa.display
from tqdm import tqdm
import os

In [3]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model
import pickle
from pydub import AudioSegment

C:\Users\Dell\AppData\Roaming\Python\Python37\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [4]:
def getSplitIndexes(filename):
    filename='test/fullSentence/'+filename
    
    def isNaN(x):
        return x != x
    signal,sample_rate = librosa.load(filename)
    
    for i in range(len(signal)):
        if signal[i] < 0:
            signal[i] = -signal[i]
    
    index_list = []
    j = 0
    while j < len(signal):
        index_list.append(j)
        j = j + 1
        
    df = pd.DataFrame(list(zip(index_list, signal)),columns =['time', 'signal'])
    
    df['mov_avg'] = df['signal'].rolling(10000).sum()
    
    smooth_list = df['mov_avg'].tolist()
    #print(len(smooth_list))
    
    minimas = []
    i = 1
    while isNaN(smooth_list[i]):
        i = i + 1
    
    #print(i)
    window = 15000

    while i + window < len(smooth_list):
        #print("hi")
        m = min(smooth_list[i:i+window])
        idx = smooth_list.index(m)
        #print(m,idx,smooth_list[i],smooth_list[i+window-1])
        
        if m != smooth_list[i] and m != smooth_list[i+window-1]:
            if len(minimas) != 0  and abs(idx - minimas[-1][0]) > 5000:  
                minimas.append([idx,m])
            if len(minimas) == 0:
                minimas.append([idx,m])
    
        i = i + window

    print(minimas)
    return minimas

In [5]:
def splitWaveFile(fileName):
    splitIndexes = getSplitIndexes(fileName)
    signal,sample_rate = librosa.load('test/fullSentence/'+fileName)
    splitIndexes.insert(0, [0,0])
    for i in range(1,len(splitIndexes)):
        subSignal=signal[splitIndexes[i-1][0]:splitIndexes[i][0]]
        subSignal = np.array(subSignal* (1<<15), dtype=np.int16)
        audio_segment = AudioSegment(
            subSignal.tobytes(), 
            frame_rate=sample_rate,
            sample_width=subSignal.dtype.itemsize, 
            channels=1
        )
        audio_segment.export("test/original/"+fileName.split(".")[0]+"_"+str(i)+".wav",format="wav")
    
    subSignal=signal[splitIndexes[-1][0]:]
    subSignal = np.array(subSignal* (1<<15), dtype=np.int16)
    audio_segment = AudioSegment(
            subSignal.tobytes(), 
            frame_rate=sample_rate,
            sample_width=subSignal.dtype.itemsize, 
            channels=1
    )
    audio_segment.export("test/original/"+fileName.split(".")[0]+"_"+str(len(splitIndexes))+".wav",format="wav")
    return len(splitIndexes)

In [6]:
def preProcess(fileName):
    def originalToScaled(fileName):
        signal,sample_rate=librosa.load("test/original/"+fileName)
        obsMax = max(signal)
        obsMin = 0

        newMax = 0.1
        newMin = 0

        new_signal = []

        for ele in signal:
            ele = abs(ele)
            x = (newMax - newMin)/(obsMax - obsMin)*(ele - obsMin) + newMin
            new_signal.append(x)

        new_signal = np.array(new_signal)

        for i in range(len(signal)):
            if signal[i] < 0:
                new_signal[i] = 0 - new_signal[i]

        new_signal = np.array(new_signal* (1<<15), dtype=np.int16)

        audio_segment = AudioSegment(
        new_signal.tobytes(), 
        frame_rate=sample_rate,
        sample_width=new_signal.dtype.itemsize, 
        channels=1
        )

        audio_segment.export("test/scaled/"+fileName,format="wav")
        
    
    def scaledToTrimmed(fileName):
        def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
            '''
            sound is a pydub.AudioSegment
            silence_threshold in dB
            chunk_size in ms

            iterate over chunks until you find the first one with sound
            '''
            trim_ms = 0 # ms

            assert chunk_size > 0 # to avoid infinite loop
            while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
                trim_ms += chunk_size

            return trim_ms

        sound = AudioSegment.from_file("test/scaled/"+fileName, format="wav")

        start_trim = detect_leading_silence(sound)
        end_trim = detect_leading_silence(sound.reverse())

        duration = len(sound)    
        trimmed_sound = sound[start_trim:duration-end_trim]
        trimmed_sound.export("test/trimmed/"+fileName, format="wav")

    def trimmedToSpeedChange(fileName):
        targetLen=20000
        sound = AudioSegment.from_file("test/trimmed/"+fileName)
        sample = sound.get_array_of_samples()
        sample = np.array(sample)
        sampleLen=len(sample)

        def speed_change(sound, speed):

            sound_with_altered_frame_rate = sound._spawn(sound.raw_data, overrides={
                "frame_rate": int(sound.frame_rate * speed)
            })
            sound_with_altered_frame_rate.export('test/speedChange/'+fileName, format ="wav")

        slow_sound = speed_change(sound, sampleLen/targetLen)
    
    originalToScaled(fileName)
    scaledToTrimmed(fileName)
    trimmedToSpeedChange(fileName)

In [7]:
def wordPredictor(sentence):
    sol = []
    if len(sentence.split()) == 1:
        encoded_text = tokenizerOne.texts_to_sequences([sentence])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=1, truncating='pre')
        print(encoded_text, pad_encoded)
        for i in (modelOne.predict(pad_encoded)[0]).argsort()[-7:][::-1]:
          pred_word = tokenizerOne.index_word[i]
          sol.append(pred_word)
            
    elif len(sentence.split()) == 2:
        encoded_text = tokenizerTwo.texts_to_sequences([sentence])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=2, truncating='pre')
        print(encoded_text, pad_encoded)
        for i in (modelTwo.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
          pred_word = tokenizerTwo.index_word[i]
          sol.append(pred_word)
            
    else:
        encoded_text = tokenizerThree.texts_to_sequences([sentence])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=3, truncating='pre')
        print(encoded_text, pad_encoded)
        for i in (modelThree.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
          pred_word = tokenizerThree.index_word[i]
          sol.append(pred_word)
            
    return sol

In [8]:
def predict(chunk_data,classes,actualWord):
    
    df = pd.read_csv(r"references.csv")
    print("classes = ", classes)
    if(actualWord not in classes):
        classes.append(actualWord)
    audio1 = getMovingAvg(chunk_data)
    similarities = {}
    len1 = len(audio1)
    sList = []
    for key in moving_avg.keys():
        if key in classes:
            if key not in similarities:
                similarities[key] = []

            for audio2 in moving_avg[key]:
                len2 = len(audio2)
                if(len1 >= len2):
                    temp1 = copy.deepcopy(audio1)
                    temp2 = copy.deepcopy(audio2)
                else:
                    temp1 = copy.deepcopy(audio2)
                    temp2 = copy.deepcopy(audio1)

                while len(temp1) > len(temp2):
                    temp1.pop()

                S = dot(temp1, temp2)/(norm(temp1)*norm(temp2))
                similarities[key].append(S)

                sList.append([key,S])
        
        
    for k,v in similarities.items():
        similarities[k] = sum(v)/len(v)
    
    similarityOutput=max(similarities.items(), key=operator.itemgetter(1))[0]
    similarities={k: v for k, v in sorted(similarities.items(), key=lambda item: item[1],reverse=True)}
    print(similarities)

    sList = sorted(sList, key = lambda x: x[1], reverse = True)
#     for i in sList[:20]:
#         print(i)
        
    sList = sList[:20]
    output_dic = {}
    for i in sList:
        if i[0] not in output_dic.keys():
            output_dic[i[0]] = 0
        output_dic[i[0]] += 1
    output_dic={k: v for k, v in sorted(output_dic.items(), key=lambda item: item[1],reverse=True)}
    print("Before weights",output_dic)
    
    
    weights=[1.0]
    for i in range(len(classes)-1):
        weights.insert(0,weights[0]+0.2)
        
    for k,v in output_dic.items():
        output_dic[k] = output_dic[k]*(weights[classes.index(k)])
    output_dic={k: v for k, v in sorted(output_dic.items(), key=lambda item: item[1],reverse=True)}
    print("After weights,",output_dic)
    output = max(output_dic.items(), key=operator.itemgetter(1))[0]
    output_dic=sorted(output_dic.items(), key=lambda item: item[1])
    
    return [similarityOutput,output]
    


In [9]:
modelOne = load_model("../wordPredictor/modelOne/modelOne.h5")
modelTwo = load_model("../wordPredictor/modelTwo/modelTwo.h5")
modelThree = load_model("../wordPredictor/modelThree/modelThree.h5")

In [10]:
with open('../wordPredictor/modelOne/tokenizerOne.tokenizer', 'rb') as config_dictionary_file:
    tokenizerOne = pickle.load(config_dictionary_file)

with open('../wordPredictor/modelTwo/tokenizerTwo.tokenizer', 'rb') as config_dictionary_file:
    tokenizerTwo = pickle.load(config_dictionary_file)

with open('../wordPredictor/modelThree/tokenizerThree.tokenizer', 'rb') as config_dictionary_file:
    tokenizerThree = pickle.load(config_dictionary_file)



In [11]:
with open('moving_avg.dictionary', 'rb') as config_dictionary_file:
    moving_avg = pickle.load(config_dictionary_file)

In [12]:
averageAccuracy=[]

In [13]:
def getMovingAvg(signal):
    #signal,sample_rate = librosa.load(filename)
    #librosa.display.waveplot(signal,sr=sample_rate)
    
    for i in range(len(signal)):
        if signal[i] < 0:
            signal[i] = -signal[i]
        
    index_list = []
    j = 0
    while j < len(signal):
        index_list.append(j)
        j = j + 1
        
    df = pd.DataFrame(list(zip(index_list, signal)),columns =['time', 'signal'])
    
    '''sns.set(rc={'figure.figsize':(20,8.27)})
    sns.lineplot(x='time',y='signal',data= df)
    plt.show()'''
    
    df['mov_avg'] = df['signal'].rolling(1500).sum()
    
    '''sns.set(rc={'figure.figsize':(20,8.27)})
    sns.lineplot(x='time',y='mov_avg',data= df)
    plt.show()'''
    #print(df['mov_avg'][1499:])
    smooth_list = df['mov_avg'][1499:].to_list()
    #print(smooth_list)
    return smooth_list

In [14]:
def speech_to_text(filename,totalFiles):
    df = pd.read_csv("DL_sample_sentences.csv")
    ID = filename[3:]
    ID = ID[:-4]
    index = df["ID"].to_list().index(ID)
    actualSentence = df["Mixed"].to_list()[index].lower()
    print("actual_sentence = ",actualSentence)
    cur_sentence = "s"
    actualSentence = actualSentence.split()
    simPS = ""
    weightPS=" "
    print(totalFiles)
    for i in range(1,totalFiles+1):
        X,sample_rate=librosa.load('test/speedChange/'+fileName.split(".")[0]+"_"+str(i)+".wav" , res_type = 'kaiser_fast')
        classes = wordPredictor(cur_sentence)
        funcOutput=predict(X,classes,actualSentence[i-1])
        simNextWord=funcOutput[0].lower()
        weightNextWord=funcOutput[1].lower()
        print("Highest Average Similarity Predicted Word ",simNextWord)
        print("Highest Frequency Predicted Word ",weightNextWord)
        simPS +=  " " + simNextWord
        weightPS +=  " " + weightNextWord
        cur_sentence = cur_sentence + " " + actualSentence[i-1]
        
    print("Highest Average Similarity Predicted Sentence ",simPS)
    print("Highest Frequency Sentence ",weightPS)

    return cur_sentence

In [15]:
fileName = 'ad_58_1.wav'   
totalFiles=splitWaveFile(fileName)

[[37809, 13.05748274519064], [65385, 12.928407609132792], [89288, 14.55607221989402]]


In [16]:
for i in range(1,totalFiles+1):
    preProcess(fileName.split(".")[0]+"_"+str(i)+".wav")

In [17]:
final_output = speech_to_text(fileName,totalFiles)
print(final_output)

actual_sentence =  nanna next turn yenu
4
[1] [[1]]
classes =  ['what', 'hathira', 'hathiradha', 'are', 'nanna', 'how', 'bheti']
{'nanna': 0.909999698361957, 'are': 0.8684239185365122, 'hathira': 0.8429909782236167, 'how': 0.8398534559567687, 'hathiradha': 0.8204536981585854, 'bheti': 0.772599791514836, 'what': 0.7635959130542181}
Before weights {'nanna': 18, 'are': 2}
After weights, {'nanna': 25.2, 'are': 3.1999999999999997}
Highest Average Similarity Predicted Word  nanna
Highest Frequency Predicted Word  nanna
[1, 21] [[ 1 21]]
classes =  ['hathira', 'next', 'traffic']
{'next': 0.8885499415416608, 'traffic': 0.8581023125049895, 'hathira': 0.5544009349055259}
Before weights {'next': 15, 'traffic': 5}
After weights, {'next': 18.0, 'traffic': 5.0}
Highest Average Similarity Predicted Word  next
Highest Frequency Predicted Word  next
[1, 30, 31] [[ 1 30 31]]
classes =  ['turn', 'places', 'plumbing']
{'turn': 0.9567368809176188, 'plumbing': 0.942401906497795, 'places': 0.8536701122846241